## Dependencies

In [88]:
# Confirmed needed dependencies
import random
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim

# Dependencies for Zillow data
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults
# zillow id, needed for the GetUpdatedPropertyDetails
from pyzillow.pyzillow import ZillowWrapper, GetUpdatedPropertyDetails


# Research how to make config.py truly invisible on GitHub Repo
from config import Zseth, Zkat, Zyuta
# Paste in your zwsid in the meantime using the following line
# zwsid = ""



# In README give instructions on setting config file and needed variables for project

## File inputs/outputs

In [15]:
randLatLon_csv = "./Data/randomLatLon.csv" 
addressList_csv = "./Data/addressList.csv"
masterData_csv = "./Data/masterData.csv"

In [3]:
# Valerie

# Further setting up Notebook with Markdown sections, file outputs, action comments, etc.
# Create pandas dataframe 

## Geocoordinates of Austin

In [4]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [5]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 10         #### Set radius in miles

points = 1000        #### Set number of lat,lon points to generate

In [6]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points = points+1
#####this can be written more succinctly as points += 1

In [7]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

1000

In [8]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map

Troy is working on this as a way to verify good random sampling of coordinate points and area selection, and as a way to get familar with plotting on maps


In [9]:
gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

#gmap.plot(latitudes, longitudes, 'cornflowerblue', edge_width=10)
gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"], '#3B0B39', size=10, marker=False)
#gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"] , 'k', marker=True)
#gmap.heatmap(, heat_lngs)

gmap.draw("./Visuals/mymap.html")

## Convert Coordinates to Residential Addresses

In [10]:
#########################
##### Yuta's Blocks #####
#########################


##### General Analysis on the Geopy Nominatim Method Below #####
# - You can filter the address by type='house', but some houses are labeled with type='yes'
# - 'Yes' type means means it's an unlabeled building.
# - Quite a bit of 'yes' type buildings actually hit a residential building, so we may get a high percentage of
#   residential buildings if we filter by 'house' or 'yes'.
# - Out of roughly 100 requests, around 70% will result in either 'house' or 'yes' type.
# - I can't seem to find the limit of API request, but we are throttled to 1 request per second.

# Testing Geopy Nominatim API Response


geopy.geocoders.options.default_user_agent = "my-application"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

# Params 1 - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== #01 - Home Response:")
response_1 = requests.get(url, params=params_1).json()
pp.pprint(response_1)
print("\n" + "="*60 + "\n")

===== #01 - Home Response:
{   'address': {   'country': 'USA',
                   'country_code': 'us',
                   'county': 'Williamson County',
                   'hamlet': 'Pond Springs',
                   'house_number': '12126',
                   'neighbourhood': "Hunter's Chase",
                   'postcode': '78729',
                   'road': 'Hunters Chase Drive',
                   'state': 'Texas',
                   'suburb': 'Jollyville'},
    'addresstype': 'place',
    'boundingbox': ['30.440962', '30.441162', '-97.777478', '-97.777278'],
    'category': 'place',
    'display_name': "12126, Hunters Chase Drive, Hunter's Chase, Jollyville, "
                    'Pond Springs, Williamson County, Texas, 78729, USA',
    'importance': '0',
    'lat': '30.441062',
    'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
               'https://osm.org/copyright',
    'lon': '-97.777378',
    'name': None,
    'osm_id': '3825454876',
    'osm_type': 'node',
 

In [11]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

,lat,lon
0,30.354028,-97.666848
1,30.341459,-97.786288
2,30.272466,-97.598273
3,30.175445,-97.824254
4,30.246398,-97.645923


In [12]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

# Might not need to do this step, see below

In [13]:
# Loop Request API and append to create new dataframe
query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
# Verify this field is unneeded, then delete if not
# aType = []
lat = []
lon = []
neighborhood = []
# Verify this field is unneeded, then delete if not
# addType = []

counter = 1
numRequests = len(lats)

print(f"Processing {numRequests} Requests...")


# Don't actually need to zip lat and lons for this
# can use "for lat_lon in latlon_df.itertuples():"
# Check into this and see if you can cut out the zip part
for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }
# Check to see if you can do multiple queries at the same time to make this faster
# i.e. send batch requests/list of queries rather than loop of single queries
    response = requests.get(query_url, params=params).json()
    
    if response['type'] == 'house' or response['type'] == 'yes':
        postcode.append(response['address']['postcode'])
#         Do we need to store aType?
#         aType.append(response['type'])
        lat.append(response['lat'])
        lon.append(response['lon'])
    
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
            
# Verify this field is unneeded, then delete if not
#         try:
#             addType.append(response['addresstype'])
#         except (KeyError, IndexError):
#             addType.append("NA")


# Printing to console is good for development to see what is happening, but takes a long time.  
# Instead of printing each line to the console, can you store these in two lists (success and failure) 
# that we could run len() on to get counts for stats later?
        print(f"Processed Record {counter} of {numRequests}.")
#         counter = counter + 1
        counter += 1
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
#         counter = counter + 1
        counter += 1

Processing 1000 Requests...
Processed Record 1 of 1000.
Processed Record 2 of 1000.
Wrong Type - Skipped Record 3 of 1000.
Processed Record 4 of 1000.
Processed Record 5 of 1000.
Processed Record 6 of 1000.
Processed Record 7 of 1000.
Processed Record 8 of 1000.
Processed Record 9 of 1000.
Processed Record 10 of 1000.
Processed Record 11 of 1000.
Processed Record 12 of 1000.
Wrong Type - Skipped Record 13 of 1000.
Processed Record 14 of 1000.
Processed Record 15 of 1000.
Processed Record 16 of 1000.
Processed Record 17 of 1000.
Processed Record 18 of 1000.
Processed Record 19 of 1000.
Processed Record 20 of 1000.
Processed Record 21 of 1000.
Processed Record 22 of 1000.
Wrong Type - Skipped Record 23 of 1000.
Processed Record 24 of 1000.
Processed Record 25 of 1000.
Wrong Type - Skipped Record 26 of 1000.
Processed Record 27 of 1000.
Processed Record 28 of 1000.
Processed Record 29 of 1000.
Wrong Type - Skipped Record 30 of 1000.
Wrong Type - Skipped Record 31 of 1000.
Wrong Type - Ski

Wrong Type - Skipped Record 246 of 1000.
Wrong Type - Skipped Record 247 of 1000.
Processed Record 248 of 1000.
Processed Record 249 of 1000.
Wrong Type - Skipped Record 250 of 1000.
Processed Record 251 of 1000.
Processed Record 252 of 1000.
Wrong Type - Skipped Record 253 of 1000.
Processed Record 254 of 1000.
Processed Record 255 of 1000.
Wrong Type - Skipped Record 256 of 1000.
Processed Record 257 of 1000.
Processed Record 258 of 1000.
Processed Record 259 of 1000.
Processed Record 260 of 1000.
Processed Record 261 of 1000.
Processed Record 262 of 1000.
Wrong Type - Skipped Record 263 of 1000.
Wrong Type - Skipped Record 264 of 1000.
Processed Record 265 of 1000.
Wrong Type - Skipped Record 266 of 1000.
Wrong Type - Skipped Record 267 of 1000.
Wrong Type - Skipped Record 268 of 1000.
Processed Record 269 of 1000.
Processed Record 270 of 1000.
Wrong Type - Skipped Record 271 of 1000.
Processed Record 272 of 1000.
Processed Record 273 of 1000.
Processed Record 274 of 1000.
Wrong Typ

Wrong Type - Skipped Record 489 of 1000.
Processed Record 490 of 1000.
Processed Record 491 of 1000.
Processed Record 492 of 1000.
Processed Record 493 of 1000.
Processed Record 494 of 1000.
Wrong Type - Skipped Record 495 of 1000.
Wrong Type - Skipped Record 496 of 1000.
Processed Record 497 of 1000.
Processed Record 498 of 1000.
Processed Record 499 of 1000.
Wrong Type - Skipped Record 500 of 1000.
Wrong Type - Skipped Record 501 of 1000.
Wrong Type - Skipped Record 502 of 1000.
Processed Record 503 of 1000.
Processed Record 504 of 1000.
Processed Record 505 of 1000.
Processed Record 506 of 1000.
Processed Record 507 of 1000.
Processed Record 508 of 1000.
Processed Record 509 of 1000.
Wrong Type - Skipped Record 510 of 1000.
Processed Record 511 of 1000.
Processed Record 512 of 1000.
Wrong Type - Skipped Record 513 of 1000.
Processed Record 514 of 1000.
Wrong Type - Skipped Record 515 of 1000.
Processed Record 516 of 1000.
Processed Record 517 of 1000.
Processed Record 518 of 1000.
W

Wrong Type - Skipped Record 732 of 1000.
Processed Record 733 of 1000.
Processed Record 734 of 1000.
Wrong Type - Skipped Record 735 of 1000.
Processed Record 736 of 1000.
Wrong Type - Skipped Record 737 of 1000.
Processed Record 738 of 1000.
Processed Record 739 of 1000.
Wrong Type - Skipped Record 740 of 1000.
Wrong Type - Skipped Record 741 of 1000.
Processed Record 742 of 1000.
Processed Record 743 of 1000.
Processed Record 744 of 1000.
Wrong Type - Skipped Record 745 of 1000.
Processed Record 746 of 1000.
Wrong Type - Skipped Record 747 of 1000.
Processed Record 748 of 1000.
Wrong Type - Skipped Record 749 of 1000.
Wrong Type - Skipped Record 750 of 1000.
Wrong Type - Skipped Record 751 of 1000.
Processed Record 752 of 1000.
Processed Record 753 of 1000.
Wrong Type - Skipped Record 754 of 1000.
Wrong Type - Skipped Record 755 of 1000.
Processed Record 756 of 1000.
Wrong Type - Skipped Record 757 of 1000.
Processed Record 758 of 1000.
Processed Record 759 of 1000.
Wrong Type - Skip

Processed Record 974 of 1000.
Wrong Type - Skipped Record 975 of 1000.
Wrong Type - Skipped Record 976 of 1000.
Wrong Type - Skipped Record 977 of 1000.
Processed Record 978 of 1000.
Processed Record 979 of 1000.
Processed Record 980 of 1000.
Wrong Type - Skipped Record 981 of 1000.
Processed Record 982 of 1000.
Wrong Type - Skipped Record 983 of 1000.
Processed Record 984 of 1000.
Processed Record 985 of 1000.
Processed Record 986 of 1000.
Processed Record 987 of 1000.
Processed Record 988 of 1000.
Wrong Type - Skipped Record 989 of 1000.
Wrong Type - Skipped Record 990 of 1000.
Processed Record 991 of 1000.
Processed Record 992 of 1000.
Wrong Type - Skipped Record 993 of 1000.
Wrong Type - Skipped Record 994 of 1000.
Processed Record 995 of 1000.
Processed Record 996 of 1000.
Processed Record 997 of 1000.
Processed Record 998 of 1000.
Processed Record 999 of 1000.
Wrong Type - Skipped Record 1000 of 1000.


In [14]:
# Create a dataframe with all addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
#     "type": aType,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
#     "address type": addType
})
address_df

# drop type and address type, or rather don't gather in the first place

,house #,street,zipcode,lat,lon,neighborhood
0,9807,Brown Lane,78754,30.3543415,-97.6671088104004,Cameron Industrial
1,4804,Precipice Cove,78731,30.3416515,-97.7854097930036,Cat Mountain
2,2110,West Slaughter Lane,78748,30.175305,-97.8250098584229,Ford Oaks
3,8920,Ramirez Lane,78742,30.2457912,-97.6477541,NA
4,11266,Taylor Draper Lane,78759,30.4104765,-97.75293440625,Angus Valley
5,904,West 30th Street,78705,30.298795,-97.7459816846154,Central Austin
6,6704,Rockridge Court,78744,30.18322,-97.7564654403255,Indian Hills
7,3804,Breckenridge Drive,00787,30.20012,-97.697807,NA
8,1500,Montopolis Drive,78741,30.228755,-97.701353,Montopolis
9,8523,Connor Lane,78753,30.345091,-97.682257,Heritage Hills


In [15]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [16]:
# Pull from CSV
address_sample = pd.read_csv(addressList_csv)
address_sample.head()

,house #,street,zipcode,lat,lon,neighborhood
0,9807,Brown Lane,78754,30.354341,-97.667109,Cameron Industrial
1,4804,Precipice Cove,78731,30.341652,-97.785410,Cat Mountain
2,2110,West Slaughter Lane,78748,30.175305,-97.825010,Ford Oaks
3,8920,Ramirez Lane,78742,30.245791,-97.647754,NaN
4,11266,Taylor Draper Lane,78759,30.410477,-97.752934,Angus Valley


In [17]:
# Map out the sample with gmplot

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(address_sample["lat"], address_sample["lon"], 'red', size=80, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Zillow API Calls using Address and Zipcode

In [74]:
# Val create function for read in csv, check for headers?, and append csv
address_sample = pd.read_csv(addressList_csv)
address_df = pd.DataFrame(address_sample)
address_df.head()
# masterData.csv
address = zip(address_df["house #"], address_df["street"])
len(address_df)

651

In [76]:
################# NO LONGER USING THIS API #################
########### HOWEVER, CODE MAY BE USED ELSEWHERE ###########

# pp = pprint.PrettyPrinter(indent=4)
# api = zillow.ValuationApi()
# # Insert your Zillow API key here
# zwsid = "X1-ZWz1gm14kn7d3f_2kpit"

# valuation = []
# sqft = []

# for row, home in address_df.iterrows():
#     print(row)
#     address = address_df["house #"][row] + " " + address_df["street"][row] 
#     print(address)
#     zipcode = address_df["zipcode"][row]
#     print(zipcode)
    
#     try:
#         z_deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
#         print(z_deep_results)
#         pp.pprint(deep_results.get_dict())
#         print(z_deep_results['zestimate']['amount'])
#         valuation.append(z_deep_results['zestimate']['amount'])
#         print(z_deep_results['extended_data']['finished_sqft'])
#         sqft.append(z_deep_results['extended_data']['finished_sqft'])
#     except:
#         print(f"No record found for {address}, {zipcode}.")



In [90]:
address = "4804 Precipice Cove"
zipcode = "78731"

zid = []
valuation = []
sqft = []
isd = []


##### VALERIE STILL WORKING ON THE LOOPING FUNCTION OF THIS #####
###### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY ######

# for row, home in address_df.iterrows():
#     print(row)
#     print(home)
# address = str(address_df["house #"][1] + " " + address_df["street"][1] )
# print(address)
# zipcode = str(address_df["zipcode"][1])
# print(zipcode)

# print(f"No record found for {address}, {zipcode} (index {row}).")

zillow_data = ZillowWrapper(Zyuta)
deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
result = GetDeepSearchResults(deep_search_response)


try:
    zillowID = result.zillow_id
    zid.append(result.zillow_id)
except:
    zid.append(None)

try:    
    val = result.zestimate_amount
    valuation.append(val)
except:
    valuation.append(None)

try:
    zsqft = result.home_size
    sqft.append(zsqft)
except:
    sqft.append(None)

updated_property_details_response = zillow_data.get_updated_property_details(zillowID)
new_result = GetUpdatedPropertyDetails(updated_property_details_response)

try:
    zisd = new_result.school_district
    isd.append(zisd)
except:
    isd.append(None)

In [91]:
print(valuation)

['2507628']


In [7]:
# Do not run yet

# masterDF = pd.DataFrame({
#     "address": house_num,
#     "zipcode": postcode,
#     "alat": lat,
#     "alon": lon,
#     "neighborhood": neighborhood,
# })

[]


## Calculate Value per Sqft

In [ ]:
#Create function to calculate value per sqft using "zestimate"["amount"] / "extended_data"["finished_sqft"]


## Crime Data

In [ ]:
# Kat's section

## School Data

In [ ]:
# Seth's section